In [1]:
from pymongo import MongoClient
import requests
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
class WebScraping:
    headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15"}
    content=""
    db=np.nan
    def __init__(self): 
        klient=MongoClient("mongodb://127.0.0.1:27017")
        self.db=klient['N5205_2024'] 
    def pobierz(self,url):
        r=requests.get(url,headers=self.headers,verify=False)
        self.content=r.text
        strona={'url':url,'contents':r.text,'date':datetime.now()}
        rezultat=self.db.proj.insert_one(strona) 
        print("Wstawiłem stronę",url,"z numerem",rezultat.inserted_id)
    def atrybuty(self,url):
        contents=self.db.proj.find_one({'url':url})
        soup=BeautifulSoup(contents['contents'],'html.parser')
        hotele = soup.findAll('div', {'data-testid': 'property-card'})
        hotele_data = []
        for hotel in hotele:
            nazwa_element = hotel.find('div', {'data-testid': 'title'})
            nazwa = nazwa_element.text.strip()
            lokacja_element = hotel.find('span', {'data-testid': 'address'})
            lokacja = lokacja_element.text.strip()
            cena_element = hotel.find('span', {'data-testid': 'price-and-discounted-price'})
            cena = cena_element.text.strip()
            hotele_data.append({
                'nazwa': nazwa,
                'lokacja': lokacja,
                'cena': cena
            })

    def zapisz(self,nazwaPliku):
        dane=self.db.proj.find({})
        for d in dane:
            soup=BeautifulSoup(d['contents'],'html.parser')
            hotele = soup.findAll('div', {'data-testid': 'property-card'})
            hotele_data = []
            for hotel in hotele:
                nazwa_element = hotel.find('div', {'data-testid': 'title'})
                nazwa = nazwa_element.text.strip()
                lokacja_element = hotel.find('span', {'data-testid': 'address'})
                lokacja = lokacja_element.text.strip()
                cena_element = hotel.find('span', {'data-testid': 'price-and-discounted-price'})
                cena = cena_element.text.strip()
                hotele_data.append({
                    'nazwa': nazwa,
                    'lokacja': lokacja,
                    'cena': cena
                })
            df=pd.DataFrame(hotele_data)
            df.to_csv(nazwaPliku+".csv",index=False,sep=";")
            df.to_excel(nazwaPliku+".xlsx",index=False)
            
    def zapiszJSON(self,nazwaPliku):
        dane=self.db.proj.find({})
        for d in dane:
            soup=BeautifulSoup(d['contents'],'html.parser')
            hotele = soup.findAll('div', {'data-testid': 'property-card'})
            hotele_data = []
            for hotel in hotele:
                nazwa_element = hotel.find('div', {'data-testid': 'title'})
                nazwa = nazwa_element.text.strip()
                lokacja_element = hotel.find('span', {'data-testid': 'address'})
                lokacja = lokacja_element.text.strip()
                cena_element = hotel.find('span', {'data-testid': 'price-and-discounted-price'})
                cena = cena_element.text.strip()
                hotele_data.append({
                    'nazwa': nazwa,
                    'lokacja': lokacja,
                    'cena': cena
                })
            df=pd.DataFrame(hotele_data)
            df.to_json(nazwaPliku+".json",orient='records')

/Users/U756317/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
w=WebScraping()
w.pobierz("https://www.booking.com/searchresults.en-gb.html?ss=Poznań&ssne=Poznań&ssne_untouched=Poznań&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaLYBiAEBmAEJuAEHyAEN2AEB6AEBiAIBqAIDuAKYjrizBsACAdICJDNlOGMyYTZkLTA1OTktNDU5NC1iZDI4LTIwZGRiMTJmMGQ3ZdgCBeACAQ&sid=05ba7057eee40dda6cb3fec45d21d6c2&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=-523642&dest_type=city&checkin=2024-07-24&checkout=2024-07-25&group_adults=2&no_rooms=1&group_children=0")

/Users/U756317/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.booking.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Wstawiłem stronę https://www.booking.com/searchresults.en-gb.html?ss=Poznań&ssne=Poznań&ssne_untouched=Poznań&efdco=1&label=gen173nr-1BCAEoggI46AdIM1gEaLYBiAEBmAEJuAEHyAEN2AEB6AEBiAIBqAIDuAKYjrizBsACAdICJDNlOGMyYTZkLTA1OTktNDU5NC1iZDI4LTIwZGRiMTJmMGQ3ZdgCBeACAQ&sid=05ba7057eee40dda6cb3fec45d21d6c2&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=-523642&dest_type=city&checkin=2024-07-24&checkout=2024-07-25&group_adults=2&no_rooms=1&group_children=0 z numerem 666e0dcefdc9492b38db0b83


In [5]:
w.db.proj.count_documents({})

1

In [6]:
w.zapiszJSON("projekt_hotele")

In [7]:
w.zapisz("projekt_hotele")

In [14]:
df=pd.read_excel('projekt_hotele.xlsx')
print('liczba wierszy',df.shape[0])
print(df['nazwa'].groupby(df['nazwa']).count())
print(df['lokacja'].groupby(df['lokacja']).count())

liczba wierszy 25
nazwa
Apartamenty Gołębia-Budget                 1
B&B Hotel Poznań Old Town                  1
Comfortable rooms                          1
Cooltour                                   1
Economic Mandala Apartment                 1
Erasmus Rooms Genius- 24h self check-in    1
Fortune Old Town boutique hotel            1
Glam House Apartments                      1
Gołębia 4                                  1
Ionia Apartment - Stary Rynek-             1
Kozia 24/25                                1
Kozia Store- Stary Rynek                   1
Lilia House                                1
Mini Apartamenty Poznań                    1
Modern&Cozy Rooms                          1
Old Town Mystery-                          1
PURO Hotel Poznań Stare Miasto             1
Plac Wolności 6                            1
Rynek 11                                   1
Rynek 12                                   1
Rynek 5                                    1
Rynek 6                        